In [1]:
import numpy as np 
import pulp
import matplotlib.pyplot as plt
import seaborn as sns

In [54]:
# Paramètres
N = 10  # Nombre de points de collecte (sans compter le dépôt)
K = 2  # Nombre de camions
Q = 15  # Capacité des camions en tonnes
M = 1000  # Grande constante pour les contraintes de temps



In [ ]:
# Demandes de collecte pour chaque point (y compris le dépôt qui est 0)
demandes = [0,2, 4, 3, 5, 2, 3, 4, 1, 5, 2]  # en tonnes
early_times = [0, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]  # Dépôt (index 0) et points de collecte
late_times = [10, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]  # Fenêtre tardive pour chaque point
service_times = [0, 1, 1, 1, 1, 1, 1,1,1,1,1]  # temps de collecte en heures

In [56]:
# Matrice de distances entre les points (symétrique)
distances = np.array([
    [0, 4, 8, 10, 7, 6, 8, 9, 5, 7, 6],
    [4, 0, 6, 8, 3, 5, 7, 6, 8, 5, 7],
    [8, 6, 0, 5, 8, 7, 6, 9, 7, 6, 8],
    [10, 8, 5, 0, 4, 3, 7, 8, 6, 7, 5],
    [7, 3, 8, 4, 0, 2, 5, 6, 7, 4, 3],
    [6, 5, 7, 3, 2, 0, 5, 6, 4, 6, 3],
    [8, 7, 6, 7, 5, 5, 0, 3, 8, 6, 4],
    [9, 6, 9, 8, 6, 6, 3, 0, 9, 8, 5],
    [5, 8, 7, 6, 7, 4, 8, 9, 0, 4, 5],
    [7, 5, 6, 7, 4, 6, 6, 8, 4, 0, 6],
    [6, 7, 8, 5, 3, 3, 4, 5, 5, 6, 0]
])

In [57]:
# Modèle de minimisation de coût
model = pulp.LpProblem("Optimisation_Collecte_Dechets", pulp.LpMinimize)


In [58]:
# Variables de décision
x = pulp.LpVariable.dicts("x", ((i, j, k) for i in range(N+1) for j in range(N+1) for k in range(K)), cat='Binary')
t = pulp.LpVariable.dicts("t", (i for i in range(N+1)), lowBound=0, cat='Continuous')


In [59]:
# Fonction objectif : minimiser la distance totale parcourue
model += pulp.lpSum(distances[i][j] * x[i, j, k] for i in range(N+1) for j in range(N+1) if i != j for k in range(K))


In [ ]:
# Contraintes

# Chaque point de collecte est visité exactement une fois
for j in range(1, N+1):
    model += pulp.lpSum(x[i, j, k] for i in range(N+1) if i != j for k in range(K)) == 1

# Contrainte de capacité pour chaque camion
for k in range(K):
    model += pulp.lpSum(demandes[i] * x[i, j, k] for i in range(N+1) for j in range(1, N+1) if i != j) <= Q

# Contrainte de fenêtre de temps
for i in range(1, N+1):
    model += early_times[i] <= t[i]
    model += t[i] <= late_times[i]

# Temps d'arrivée doit respecter les temps de parcours
for i in range(N+1):
    for j in range(1, N+1):
        if i != j:
            for k in range(K):
                model += t[j] >= t[i] + service_times[i] + distances[i][j] - M * (1 - x[i, j, k])


In [61]:
# Résoudre le modèle
model.solve()

# Affichage des résultats
print("Status:", pulp.LpStatus[model.status])
print("Coût total (distance):", pulp.value(model.objective))

for k in range(K):
    print(f"\nCamion {k + 1} :")
    for i in range(N+1):
        for j in range(N+1):
            if i != j and pulp.value(x[i, j, k]) == 1:
                print(f"De {i} à {j} avec coût {distances[i][j]} et temps d'arrivée {pulp.value(t[j]):.2f}")

Status: Optimal
Coût total (distance): 50.0

Camion 1 :
De 0 à 1 avec coût 4 et temps d'arrivée 5.00
De 0 à 6 avec coût 8 et temps d'arrivée 10.00
De 0 à 8 avec coût 5 et temps d'arrivée 12.00
De 4 à 5 avec coût 2 et temps d'arrivée 12.00
De 6 à 7 avec coût 3 et temps d'arrivée 14.00

Camion 2 :
De 0 à 2 avec coût 8 et temps d'arrivée 9.00
De 0 à 3 avec coût 10 et temps d'arrivée 10.00
De 1 à 4 avec coût 3 et temps d'arrivée 9.00
De 4 à 9 avec coût 4 et temps d'arrivée 16.00
De 4 à 10 avec coût 3 et temps d'arrivée 14.00


In [65]:
import random

depot = 0  # Dépôt

# Paramètres du Recuit Simulé
initial_temperature = 1000
cooling_rate = 0.95
stopping_temperature = 1e-8
max_iterations = 1000

# Fonction de coût
def calculate_cost(solution):
    total_cost = 0
    for route in solution:
        route_cost = 0
        for i in range(len(route) - 1):
            route_cost += distances[route[i]][route[i + 1]]
        total_cost += route_cost
    return total_cost

# Fonction pour générer une solution initiale
def generate_initial_solution():
    solution = []
    nodes = list(range(1, N + 1))
    for _ in range(K):
        route = [depot]
        load = 0
        while nodes and load + demandes[nodes[0]] <= Q:
            node = nodes.pop(0)
            route.append(node)
            load += demandes[node]
        route.append(depot)
        solution.append(route)
    return solution

# Fonction pour générer une solution voisine
def generate_neighbor(solution):
    neighbor = [route[:] for route in solution]
    i, j = random.randint(0, K - 1), random.randint(1, len(neighbor[0]) - 2)
    if neighbor[i][j] != depot:
        neighbor[i][j], neighbor[(i + 1) % K][j] = neighbor[(i + 1) % K][j], neighbor[i][j]
    return neighbor

# Fonction pour exécuter le Recuit Simulé
def simulated_annealing():
    current_solution = generate_initial_solution()
    current_cost = calculate_cost(current_solution)
    best_solution = current_solution
    best_cost = current_cost
    temperature = initial_temperature

    while temperature > stopping_temperature:
        for _ in range(max_iterations):
            neighbor = generate_neighbor(current_solution)
            neighbor_cost = calculate_cost(neighbor)
            cost_diff = neighbor_cost - current_cost

            if cost_diff < 0 or np.exp(-cost_diff / temperature) > random.random():
                current_solution = neighbor
                current_cost = neighbor_cost
                if current_cost < best_cost:
                    best_solution = current_solution
                    best_cost = current_cost

        temperature *= cooling_rate
    return best_solution, best_cost

# Exécution de l'algorithme
best_solution, best_cost = simulated_annealing()
print("Meilleure solution:", best_solution)
print("Coût total:", best_cost)

Meilleure solution: [[0, 1, 2, 3, 8, 0], [0, 5, 6, 7, 4, 9, 0]]
Coût total: 57
